In [29]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

### 필요한 모듈 로딩

In [30]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

## 1. LLM을 통한 요리 정보 해석
- 이미지 -> 맛과 풍미 (image to text)
- 입력 : 요리 이미지(url)
- 출력: 요리명, 요리에 대한 풍미 설명
- 함수로 정의한 다음, RunnableLambda 객체

In [55]:
# 함수 정의 : 이미지 -> 요리명, 풍미 설명 출력
def describe_dish_flavor(input_data):

    prompt = ChatPromptTemplate([
        ("system", """
        You are a culinary expert.
        Based on the input image of a dish,

        you identify the most likely name of the dish by analyzing visual cues such as ingredients, cooking methods, and plating,

        and infer the ingredients and preparation techniques to describe the flavor profile objectively.

        If the dish name is an estimate, this should be stated clearly.
        The flavor description should focus concisely on key taste components (sweetness, saltiness, umami, richness, spiciness, acidity),
        aroma, and textural contrast.
        Subjective judgments and emotional expressions must be excluded, and the explanation should be grounded in observable facts.
        """),
        HumanMessagePromptTemplate.from_template([
            {"text": """아래 이미지의 요리에 대한 요리명과 풍미를 설명해주세요.
             출력형태 :
             요리명 :
             요리의 풍미:
            """},
            {"image_url": "{image_url}"} # image_url는 정해줘 있음.
        ])
    ])

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.1,
        api_key=GOOGLE_API_KEY
    )

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser

    return chain

In [57]:
from langchain_core.runnables import RunnableLambda

# 함수를 전달인자로 넣기
r1 = RunnableLambda(describe_dish_flavor)

# RunnableLambda를 통한 함수 실행
input_data = {
    "image_url": "https://static.wtable.co.kr/image/production/service/recipe/2068/8e90b171-4b7c-44da-affc-bb9bc8297084.jpg?size=800x800"
    }
# res = r1.invoke(input_data)

In [33]:
print(res)


요리명 : 궁중 떡볶이 (Gungjung Tteokbokki) 또는 간장 떡볶이 (Ganjang Tteokbokki) (추정)
요리의 풍미:
이 요리는 간장 기반의 양념으로 인해 짭조름하면서도 은은한 단맛과 깊은 감칠맛이 특징입니다. 소고기의 고소함과 표고버섯의 흙내음 및 진한 감칠맛이 조화를 이루며, 참기름과 깨가 더해져 고소한 향과 풍미를 더합니다. 식감은 쫄깃한 떡볶이 떡, 부드러운 소고기, 그리고 아삭한 당근의 대비로 이루어져 있습니다. 마늘과 파의 향긋함이 전체적인 풍미를 균형 있게 잡아줍니다. 매운맛은 거의 없으며, 전반적으로 풍부하고 조화로운 맛을 냅니다.



## 2.  요리에 가장 잘 어울리는 wine top 5 검색
- pinecone 백터 db 저장되어 있음
- index : wine-reviews, namespace: wine-reviews-ns

In [34]:
res = """
요리명 : 궁중 떡볶이 (Gungjung Tteokbokki) 또는 간장 떡볶이 (Ganjang Tteokbokki) (추정)
요리의 풍미:
이 요리는 간장 기반의 양념으로 인해 짭조름하면서도 은은한 단맛과 깊은 감칠맛이 특징입니다. 소고기의 고소함과 표고버섯의 흙내음 및 진한 감칠맛이 조화를 이루며, 참기름과 깨가 더해져 고소한 향과 풍미를 더합니다. 식감은 쫄깃한 떡볶이 떡, 부드러운 소고기, 그리고 아삭한 당근의 대비로 이루어져 있습니다. 마늘과 파의 향긋함이 전체적인 풍미를 균형 있게 잡아줍니다. 매운맛은 거의 없으며, 전반적으로 풍부하고 조화로운 맛을 냅니다.
"""

In [35]:

# 사용자 프롬프트 vector화, 유사도 높은 top=5 찾기
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


# 요리에 대한 풍미(설명) 들어오면,
# 벡터 db에 인덱할 때 사용한 동일 임베딩 모델을 사용해서 임베딩 벡터 생성
embedding = OpenAIEmbeddings(
     model = OPENAI_EMBEDDING_MODEL
 )

# 벡터 db에서 유사도계산, top-5 검색
# 벡터 db 객체 생성
vector_db = PineconeVectorStore(
    embedding = embedding,  # 질문에 대한 임베딩 벡터가 생성됨
    index_name = PINECONE_INDEX_NAME ,
    namespace = PINECONE_NAMESPACE
)
# 벡터 db에서 질문과 가장 유사한, top-5 검색하기
query = res  # 질문
print("질문 : ", query)
print("-"*50)
results = vector_db.similarity_search(query, k=5)  # top-5 검색

질문 :  
요리명 : 궁중 떡볶이 (Gungjung Tteokbokki) 또는 간장 떡볶이 (Ganjang Tteokbokki) (추정)
요리의 풍미:
이 요리는 간장 기반의 양념으로 인해 짭조름하면서도 은은한 단맛과 깊은 감칠맛이 특징입니다. 소고기의 고소함과 표고버섯의 흙내음 및 진한 감칠맛이 조화를 이루며, 참기름과 깨가 더해져 고소한 향과 풍미를 더합니다. 식감은 쫄깃한 떡볶이 떡, 부드러운 소고기, 그리고 아삭한 당근의 대비로 이루어져 있습니다. 마늘과 파의 향긋함이 전체적인 풍미를 균형 있게 잡아줍니다. 매운맛은 거의 없으며, 전반적으로 풍부하고 조화로운 맛을 냅니다.

--------------------------------------------------


In [36]:
# 요리에 가장 잘 어울리는 와인 5개 검색한 결과
results

[Document(id='310a50ab-e2ce-4a6d-97da-9568bc28d960', metadata={'row': 37787.0, 'source': 'c:\\JH\\ggdrive_JH\\kpmg_7th_lab\\rag_pjt_kpmg7th\\prepare\\wine_reviews\\winemag-data-130k-v2.csv'}, page_content=': 37787\ncountry: US\ndescription: Luscious in candied, pastry-filling raspberries and cherries, with dustings of cocoa, cinnamon and vanilla, and a sweet veneer of smoky sandalwood. Could be somewhat crisper and better structured, but a delicious Pinot for drinking now.\ndesignation: Meiomi\npoints: 87\nprice: 25.0\nprovince: California\nregion_1: Sonoma County-Monterey County-Santa Barbara County\nregion_2: California Other\ntaster_name: \ntaster_twitter_handle: \ntitle: Belle Glos 2008 Meiomi Pinot Noir (Sonoma County-Monterey County-Santa Barbara County)\nvariety: Pinot Noir\nwinery: Belle Glos'),
 Document(id='51838d0c-eea3-4992-a2ea-ec3b1d533b00', metadata={'row': 81971.0, 'source': 'c:\\JH\\ggdrive_JH\\kpmg_7th_lab\\rag_pjt_kpmg7th\\prepare\\wine_reviews\\winemag-data-130k-v2.

In [37]:
# "\n".join([doc.page_content for doc in results])

context = ""
for doc in results:
    # print(doc.page_content)
    context = context + doc.page_content + "\n"
    # print("-" * 50)
print(context)

: 37787
country: US
description: Luscious in candied, pastry-filling raspberries and cherries, with dustings of cocoa, cinnamon and vanilla, and a sweet veneer of smoky sandalwood. Could be somewhat crisper and better structured, but a delicious Pinot for drinking now.
designation: Meiomi
points: 87
price: 25.0
province: California
region_1: Sonoma County-Monterey County-Santa Barbara County
region_2: California Other
taster_name: 
taster_twitter_handle: 
title: Belle Glos 2008 Meiomi Pinot Noir (Sonoma County-Monterey County-Santa Barbara County)
variety: Pinot Noir
winery: Belle Glos
: 81971
country: Argentina
description: This almost translucent brick-colored Pinot offers aromas of lettuce leaf, orange peel and dried red fruits. It's light, edgy and fresh in feel, with flavors of salted meat and herbal plum. A tangy citric finish is primed with racy acidity. Fermented and aged in egg-shaped cement tanks.
designation: Eggo Filoso
points: 89
price: 30.0
province: Mendoza Province
regi

In [38]:
context = "\n".join([doc.page_content for doc in results])
print(context)

: 37787
country: US
description: Luscious in candied, pastry-filling raspberries and cherries, with dustings of cocoa, cinnamon and vanilla, and a sweet veneer of smoky sandalwood. Could be somewhat crisper and better structured, but a delicious Pinot for drinking now.
designation: Meiomi
points: 87
price: 25.0
province: California
region_1: Sonoma County-Monterey County-Santa Barbara County
region_2: California Other
taster_name: 
taster_twitter_handle: 
title: Belle Glos 2008 Meiomi Pinot Noir (Sonoma County-Monterey County-Santa Barbara County)
variety: Pinot Noir
winery: Belle Glos
: 81971
country: Argentina
description: This almost translucent brick-colored Pinot offers aromas of lettuce leaf, orange peel and dried red fruits. It's light, edgy and fresh in feel, with flavors of salted meat and herbal plum. A tangy citric finish is primed with racy acidity. Fermented and aged in egg-shaped cement tanks.
designation: Eggo Filoso
points: 89
price: 30.0
province: Mendoza Province
regi

In [56]:
# 요리에 어울리는 와인 top-5 검색결과를 리턴하는 함수 정의
def search_wines(query):
    embedding = OpenAIEmbeddings(
         model = OPENAI_EMBEDDING_MODEL
    )
    
    # 벡터 db에서 유사도계산, top-5 검색
    # 벡터 db 객체 생성
    vector_db = PineconeVectorStore(
        embedding = embedding,  # 질문에 대한 임베딩 벡터가 생성됨
        index_name = PINECONE_INDEX_NAME ,
        namespace = PINECONE_NAMESPACE
    )
    # 벡터 db에서 질문과 가장 유사한, top-5 검색하기
    results = vector_db.similarity_search(query, k=5)  # top-5 검색

    context = "\n".join([doc.page_content for doc in results])

    # 함수를 호출한 쪽으로 query, top-5의 검색 결과에 필터링한 결과를 리턴함
    return {
        "query" : query,
        "wine_reviews" : context
    }


In [40]:
# 요리에 어울리는 와인 top-5 검색결과를 리턴하는 함수 호출
query = res  # 질문
topk_results = search_wines(query)

In [41]:
topk_results["query"]

'\n요리명 : 궁중 떡볶이 (Gungjung Tteokbokki) 또는 간장 떡볶이 (Ganjang Tteokbokki) (추정)\n요리의 풍미:\n이 요리는 간장 기반의 양념으로 인해 짭조름하면서도 은은한 단맛과 깊은 감칠맛이 특징입니다. 소고기의 고소함과 표고버섯의 흙내음 및 진한 감칠맛이 조화를 이루며, 참기름과 깨가 더해져 고소한 향과 풍미를 더합니다. 식감은 쫄깃한 떡볶이 떡, 부드러운 소고기, 그리고 아삭한 당근의 대비로 이루어져 있습니다. 마늘과 파의 향긋함이 전체적인 풍미를 균형 있게 잡아줍니다. 매운맛은 거의 없으며, 전반적으로 풍부하고 조화로운 맛을 냅니다.\n'

In [43]:
topk_results["wine_reviews"]

": 37787\ncountry: US\ndescription: Luscious in candied, pastry-filling raspberries and cherries, with dustings of cocoa, cinnamon and vanilla, and a sweet veneer of smoky sandalwood. Could be somewhat crisper and better structured, but a delicious Pinot for drinking now.\ndesignation: Meiomi\npoints: 87\nprice: 25.0\nprovince: California\nregion_1: Sonoma County-Monterey County-Santa Barbara County\nregion_2: California Other\ntaster_name: \ntaster_twitter_handle: \ntitle: Belle Glos 2008 Meiomi Pinot Noir (Sonoma County-Monterey County-Santa Barbara County)\nvariety: Pinot Noir\nwinery: Belle Glos\n: 81971\ncountry: Argentina\ndescription: This almost translucent brick-colored Pinot offers aromas of lettuce leaf, orange peel and dried red fruits. It's light, edgy and fresh in feel, with flavors of salted meat and herbal plum. A tangy citric finish is primed with racy acidity. Fermented and aged in egg-shaped cement tanks.\ndesignation: Eggo Filoso\npoints: 89\nprice: 30.0\nprovince: 

### top-k 검색 함수, RunnableLambda 객체로 실행

In [45]:
from langchain_core.runnables import RunnableLambda

# 함수를 전달인자로 넣기
r2 = RunnableLambda(search_wines)

# RunnableLambda를 통한 함수 실행
input_data = query
res = r2.invoke(input_data)

In [46]:
res

{'query': '\n요리명 : 궁중 떡볶이 (Gungjung Tteokbokki) 또는 간장 떡볶이 (Ganjang Tteokbokki) (추정)\n요리의 풍미:\n이 요리는 간장 기반의 양념으로 인해 짭조름하면서도 은은한 단맛과 깊은 감칠맛이 특징입니다. 소고기의 고소함과 표고버섯의 흙내음 및 진한 감칠맛이 조화를 이루며, 참기름과 깨가 더해져 고소한 향과 풍미를 더합니다. 식감은 쫄깃한 떡볶이 떡, 부드러운 소고기, 그리고 아삭한 당근의 대비로 이루어져 있습니다. 마늘과 파의 향긋함이 전체적인 풍미를 균형 있게 잡아줍니다. 매운맛은 거의 없으며, 전반적으로 풍부하고 조화로운 맛을 냅니다.\n',
 'wine_reviews': ": 37787\ncountry: US\ndescription: Luscious in candied, pastry-filling raspberries and cherries, with dustings of cocoa, cinnamon and vanilla, and a sweet veneer of smoky sandalwood. Could be somewhat crisper and better structured, but a delicious Pinot for drinking now.\ndesignation: Meiomi\npoints: 87\nprice: 25.0\nprovince: California\nregion_1: Sonoma County-Monterey County-Santa Barbara County\nregion_2: California Other\ntaster_name: \ntaster_twitter_handle: \ntitle: Belle Glos 2008 Meiomi Pinot Noir (Sonoma County-Monterey County-Santa Barbara County)\nvariety: Pinot Noir\nwinery: Belle Glos\n: 81971\ncountry: Argentina\ndescr

### r1, r2를 파이프라인으로 실행

In [100]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(describe_dish_flavor)
r2 = RunnableLambda(search_wines)

input_data = {
    "image_url": "https://static.wtable.co.kr/image/production/service/recipe/2068/8e90b171-4b7c-44da-affc-bb9bc8297084.jpg?size=800x800"
    }

chain = r1 | r2
res2 = chain.invoke(input_data)

In [101]:
from pprint import pprint
pprint(res2)

{'query': '요리명 : 궁중 떡볶이 (Gungjung Tteokbokki)\n'
          '\n'
          '요리의 풍미:\n'
          '이 요리는 간장 기반의 양념으로 조리되어 전반적으로 짠맛과 단맛이 조화롭게 어우러지며, 소고기와 표고버섯에서 비롯된 깊은 '
          '감칠맛이 특징입니다. 참기름과 통깨의 고소한 향이 더해져 풍미를 더합니다. 식감은 쫄깃한 떡과 부드러운 소고기, 아삭한 '
          '당근, 그리고 씹는 맛이 있는 표고버섯이 조화를 이루어 다채롭습니다. 매운맛은 감지되지 않습니다.',
 'wine_reviews': ': 96906\n'
                 'country: US\n'
                 'description: Nearly garnet in color, this offers a very '
                 'strong strawberry sauce aromas, with appealing hints of '
                 'shortcake. Fruit flavors suggest pressed raspberry, but '
                 'there is a shiso leaf bitterness as well.\n'
                 'designation: \n'
                 'points: 88\n'
                 'price: 19.0\n'
                 'province: California\n'
                 'region_1: Santa Ynez Valley\n'
                 'region_2: Central Coast\n'
                 'taster_name: Matt Kettmann\n'
                 'taster_twitter_handle: @mattke

# 요리에 어울리는 와인 추천 
- 검색 결과 증강, 요청, 결과 파이프라인 함수 정의

In [ ]:
# 함수3. 요리설명, top-5의 context 입력 받고 -> 요리에 어울리는 와인 추천 
def recommand(input_data):
    prompt = ChatPromptTemplate([
        ("system", """
    🍷 Wine Sommelier – System Prompt (Short / Optimized)
    You are a professional wine sommelier specialized in food and wine pairing.

    When responding, you:
    - Analyze food characteristics (ingredients, cooking method, sauce, flavor intensity)
    - Consider wine structure (acidity, tannin, sweetness, body, alcohol)
    - Apply pairing logic (balance, contrast, complement, intensity matching)

    You always:
    - Explain why a pairing works
    - Adapt recommendations to the customer’s taste, budget, and occasion
    - Use clear, accessible language and avoid unnecessary jargon

    Your goal:
    Recommend wine pairings that create harmony between food and wine and maximize the customer’s enjoyment.
        """),
        ("human", """ 아래 와인리뷰 내용에서만 추천을 해줘
        요리 설명 : {query}
        와인 리뷰 : {wine_reviews}
        
        답변은 다음과 같이 응답해 주세요.
        추천 와인: 
        추천 이유:
        """)
    ])

    llm = ChatOpenAI(
        model='gpt-4o-mini',
        temperature=0.1,
        api_key=OPENAI_API_KEY
    )

    output_parser = StrOutputParser()

    # pipeline : 데이터의 흐름
    chain = prompt | llm | output_parser

    return chain

In [90]:
r3 = RunnableLambda(recommand)

In [91]:
# 전달해야 하는 값:
input_data = res2
res3 = r3.invoke(input_data)

In [92]:
# 와인 추천 결과
print(res3)

추천 와인: Zorzal 2015 Eggo Filoso Pinot Noir (Tupungato)

추천 이유: 궁중 떡볶이는 간장 양념의 짭짤함과 소고기, 표고버섯의 감칠맛이 조화를 이루는 요리입니다. Zorzal 2015 Eggo Filoso Pinot Noir는 가벼운 바디와 신선한 산미를 가지고 있어, 떡볶이의 풍미와 잘 어울립니다. 이 Pinot Noir는 허브와 과일의 복합적인 향이 있어, 떡볶이의 고소한 참기름과 깨의 맛을 보완해 줄 수 있습니다. 또한, 이 와인의 산미가 요리의 짭짤한 맛을 상쇄하며, 전체적인 조화를 이끌어낼 것입니다.


In [ ]:
# 함수1. 사용자는 요리 이미지 업로딩 -> 요리설명 LLM이 리턴
# 함수2. 요리설명을 입력으로 받고 -> 요리설명, top-5를 context 리턴
# 함수3. 요리설명, top-5의 context 입력 받고 -> 요리에 어울리는 와인 추천 

# chain = r1 | r2 | r3
# chain.invoke("이미지 URL")

In [102]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(describe_dish_flavor)
r2 = RunnableLambda(search_wines)
r3 = RunnableLambda(recommand)

input_data = {
    "image_url": "https://static.wtable.co.kr/image/production/service/recipe/2068/8e90b171-4b7c-44da-affc-bb9bc8297084.jpg?size=800x800"
    }

chain = r1 | r2 | r3
res4 = chain.invoke(input_data)

In [103]:
print(res4)

추천 와인: Zorzal 2015 Eggo Filoso Pinot Noir (Tupungato)

추천 이유: 궁중 떡볶이는 간장 기반의 양념으로 짠맛과 단맛이 조화롭게 어우러지며, 소고기와 표고버섯의 깊은 감칠맛이 특징입니다. 이 Pinot Noir는 가벼운 바디와 신선한 산미를 가지고 있어, 요리의 복합적인 풍미와 잘 어울립니다. 특히, 이 와인의 소금기 있는 고기 맛과 허브의 풍미가 떡볶이의 감칠맛과 조화를 이루어, 전체적인 맛의 균형을 맞춰줍니다. 또한, 와인의 상큼한 시트러스 피니시가 요리의 고소한 향미를 더욱 강조해 줄 것입니다.
